## **ResNet-34, ResNet-50 구현하기**

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

import urllib3
urllib3.disable_warnings()

/Users/yena/miniforge3/envs/py_0524_3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



![]()
<img src="https://imgur.com/8rUiKoZ.png" width="700px" height="300px" title="px(픽셀) 크기 설정" alt="Markdown Image"></img><br/>  

In [2]:
# function for building ResNet Block

def build_resnet_block(input_layer,
                    num_cnn=3, 
                    channel=64,
                    block_num=1,
                    is_50=False,
                    is_plane=False,
                   ):
    # 입력 레이어
    x = input_layer
    
    if is_50==True:
        # CNN 레이어
        for cnn_num in range(num_cnn):  # [3, 4, 6, 3]
            
            identity = x
            x = keras.layers.Conv2D(filters=channel,kernel_size=(1,1),strides=1,padding='same')(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)       
                 
            x = keras.layers.Conv2D(filters=channel,kernel_size=(3,3),strides=1,padding='same')(x)  
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)       
            
            x = keras.layers.Conv2D(filters=4*channel,kernel_size=(1,1),strides=1,padding='same')(x)
            x = keras.layers.BatchNormalization()(x)
        
            if is_plane==False:
                identity_channel = identity.shape.as_list()[-1]
                
                if identity_channel != channel:
                    identity = keras.layers.Conv2D(channel, kernel_size=(1, 1), strides=1, padding='same')(identity)
                    identity = keras.layers.BatchNormalization()(identity)
                    # skip connections
                    x = keras.layers.Add()([x, identity])
            else:
                pass    
                      
            x = keras.layers.Activation('relu')(x) 

    elif is_50==False:
        identity = x
        # CNN 레이어
        for cnn_num in range(num_cnn):
            x = keras.layers.Conv2D(filters=channel,kernel_size=(3,3),strides=1,padding='same')(x)    
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)       
            x = keras.layers.Conv2D(filters=channel,kernel_size=(3,3),strides=1,padding='same')(x)  
            x = keras.layers.BatchNormalization()(x)

            # 잔차 연결
            if is_plane==False:
                # Add를 위해 shape 맞춰주기
                identity_channel = identity.shape.as_list()[-1]
                
                if identity_channel != channel:
                    identity = keras.layers.Conv2D(channel, kernel_size=(1,1),strides=1, padding='same')(identity)
                    identity = keras.layers.BatchNormalization()(identity)
                    # skip connection
                    x = keras.layers.Add()([x, identity])
            else:
                pass
            
            x = keras.layers.Activation('relu')(x)        
                 


    return x

In [3]:
# VGG 모델 자체를 생성하는 함수입니다.
def build_resnet(input_shape=(32,32,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=10,
              is_50=False,
              is_plane=False):
    
    assert len(num_cnn_list) == len(channel_list) #모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    output = input_layer
    
    output = keras.layers.Conv2D(filters=64,kernel_size=(7,7),padding='same')(output)
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation('relu')(output)
    output = keras.layers.MaxPool2D(pool_size=(3, 3))(output)
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_resnet_block(
            output,
            num_cnn=num_cnn, 
            channel=channel,
            block_num=i,
            is_50=False,
            is_plane=False
        )
    
    output = keras.layers.AveragePooling2D()(output)
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='predictions')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

In [4]:
# 기본값 : 43
resnet_34 = build_resnet(input_shape=(32, 32, 3), is_50=False, is_plane=False)
resnet_34.summary()

2023-06-26 16:22:20.038935: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-26 16:22:20.039052: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   9472        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (

In [5]:
resnet_50 = build_resnet(input_shape=(32, 32,3), is_50=True, is_plane=False)
resnet_50.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_36 (Conv2D)             (None, 32, 32, 64)   9472        ['input_2[0][0]']                
                                                                                                  
 batch_normalization_36 (BatchN  (None, 32, 32, 64)  256         ['conv2d_36[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_33 (Activation)     (None, 32, 32, 64)   0           ['batch_normalization_36[0]

In [6]:
# plane 모델 _34
resnet_34_plain = build_resnet(input_shape=(244, 244,3), is_50=False, is_plane=False)
resnet_34_plain.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 244, 244, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_72 (Conv2D)             (None, 244, 244, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_72 (BatchN  (None, 244, 244, 64  256        ['conv2d_72[0][0]']              
 ormalization)                  )                                                           

In [7]:
# plane 모델 _50
resnet_50_plain = build_resnet(input_shape=(244, 244,3), is_50=False, is_plane=False)
resnet_50_plain.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 244, 244, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_108 (Conv2D)            (None, 244, 244, 64  9472        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_108 (Batch  (None, 244, 244, 64  256        ['conv2d_108[0][0]']             
 Normalization)                 )                                                           

## **Dataset으로 성능 비교하기**

In [8]:
# Tensorflow가 활용할 GPU가 장착되어 있는지 확인해 봅니다.
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
BATCH_SIZE = 32
EPOCH = 15
IMAGE_SIZE = 100

DATASET = https://paperswithcode.com/dataset/cats-vs-dogs  

In [10]:
# 캐글에서 직접 다운로드
dataset=tf.keras.preprocessing.image_dataset_from_directory('/Users/yena/Desktop/python_study/AIFFEL/GOINGDEEPER/dogs-vs-cats/train',
                                                            shuffle=True,
                                                            image_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                            batch_size=BATCH_SIZE)

class_names=dataset.class_names
class_names

Found 25000 files belonging to 2 classes.


['cat', 'dog']

In [11]:
dataset=dataset.take(int(len(dataset)/10))

In [12]:
train_data = dataset.take(int(len(dataset)*0.8))
val_data = dataset.take(int(len(dataset)*0.2))

train_data=train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_data=val_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [13]:
resize_and_rescale=tf.keras.Sequential([
    keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    keras.layers.experimental.preprocessing.Rescaling(1.0/255)])

data_augmentation = tf.keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.2)])

In [14]:
resnet_34 = build_resnet(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), is_50=False, is_plane=False)
resnet_34_plain = build_resnet(input_shape=(IMAGE_SIZE, IMAGE_SIZE,3), is_50=False, is_plane=False)

In [15]:
resnet_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer= keras.optimizers.Adam(),
    metrics=['accuracy'],
)

history_34 = resnet_34.fit(
    train_data,
    epochs=EPOCH,
    validation_data=val_data,
    verbose=2,
    use_multiprocessing=True,
)

Epoch 1/15


2023-06-26 16:22:23.558394: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-06-26 16:22:23.562042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-06-26 16:23:50.226626: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


62/62 - 94s - loss: 2.8485 - accuracy: 0.4879 - val_loss: 1118952.2500 - val_accuracy: 0.5042 - 94s/epoch - 2s/step
Epoch 2/15
62/62 - 93s - loss: 2.0728 - accuracy: 0.5015 - val_loss: 18195.1465 - val_accuracy: 0.4958 - 93s/epoch - 1s/step
Epoch 3/15
62/62 - 92s - loss: 1.3343 - accuracy: 0.5101 - val_loss: 83.1734 - val_accuracy: 0.5208 - 92s/epoch - 1s/step
Epoch 4/15
62/62 - 93s - loss: 0.8852 - accuracy: 0.4995 - val_loss: 1.7740 - val_accuracy: 0.5375 - 93s/epoch - 2s/step
Epoch 5/15
62/62 - 92s - loss: 0.7247 - accuracy: 0.5081 - val_loss: 0.9822 - val_accuracy: 0.5000 - 92s/epoch - 1s/step
Epoch 6/15
62/62 - 90s - loss: 0.6928 - accuracy: 0.5302 - val_loss: 0.7297 - val_accuracy: 0.5146 - 90s/epoch - 1s/step
Epoch 7/15
62/62 - 92s - loss: 0.6898 - accuracy: 0.5378 - val_loss: 0.6901 - val_accuracy: 0.5479 - 92s/epoch - 1s/step
Epoch 8/15
62/62 - 91s - loss: 0.6821 - accuracy: 0.5474 - val_loss: 0.6782 - val_accuracy: 0.5750 - 91s/epoch - 1s/step
Epoch 9/15
62/62 - 92s - loss: 0

In [ ]:
resnet_34_plain.compile(
    loss='sparse_categorical_crossentropy',
    optimizer= keras.optimizers.Adam(),
    metrics=['accuracy'],
)

history_34_plain = resnet_34_plain.fit(
    train_data,
    epochs=EPOCH,
    validation_data=val_data,
    verbose=2,
    use_multiprocessing=True,
)

In [ ]:
plt.plot(history_34.history['loss'], 'r')
plt.plot(history_34_plain.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['resnet_34', 'resnet_34_plain'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_34.history['val_accuracy'], 'r')
plt.plot(history_34_plain.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['resnet_34', 'resnet_34_plain'], loc='upper left')
plt.show()

**참고문헌**  
- He, Kaiming, et al. "Deep residual learning for image recognition." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.  
- He, Kaiming, et al. "Identity mappings in deep residual networks." Computer Vision–ECCV 2016: 14th European Conference, Amsterdam, The Netherlands, October 11–14, 2016, Proceedings, Part IV 14. Springer International Publishing, 2016.  